### __BUSA8001 (S2, 2023) Group Assignment - Predicting Used Car Sale Prices__

--- 

**Kaggle Competition Ends:** Friday, 3 November 2023 @ 3:00pm (Week 13)   

**Overview:**   

- In the group assignment you will form a team of 3 members and participate in a forecasting competition on Kaggle
- The goal is to predict prices of used cars based on car characteristics and regression models
- Assessment Summary:  
    - Write a problem statement and perform Exploratory Data Analysis  
    - Clean up data, deal with categorical features and missing observations, and create new explanatory variables (feature engineering)  
    - Construct and tune forecasting models, produce forecasts and submit your predictions to Kaggle  
    - Each member of the team will record a video presentation of their work  


---

## Task 1: Problem Description and Initial Data Analysis

1. Read the Competition Overview on Kaggle [https://www.kaggle.com/t/32b34f072642495487836cf93453ac6a](https://www.kaggle.com/t/32b34f072642495487836cf93453ac6a)
2. Referring to Competition Overview and the data provided on Kaggle write **Problem Description** (about 500 words) focusing on key points that will need to be addressed as first steps in Tasks 2 and 3 below, 

- Using the following headings:
    - Forecasting Problem - explain what you are trying to do and how it could be used in the real world (i.e. why it may be important)
    - Evaluation Criteria - explain the criteria is used to assess forecast performance 
    - Types of Variables/Features
    - Data summary and main data characteristics
    - Missing Values (only explain what you find - do not impute missing values at this stage)
    - You should **not** discuss any specific predictive algorithms at this stage
    - Note: Your written portion of this task should be completed in a single Markdown cell


##### 1. Forecasting Problem
The objective of the forecasting problem, based on the dataset provided, is to predict the listing price of a vehicle based on various features, including its specifications, condition, location, and dealer information. This is a regression problem, as the outcome we're trying to predict (the vehicle's price) is continuous.

In the real world, such a prediction model holds immense value. Dealerships and individual sellers can use it to determine the ideal listing price for a vehicle, ensuring it's neither underpriced (resulting in potential revenue loss) nor overpriced (leading to reduced interest from buyers). Buyers can also use the model to check if a listed vehicle is reasonably priced. Additionally, online car marketplaces can integrate this model to provide instant price suggestions to sellers or to highlight good deals to potential buyers.

##### 2. Evaluation Criteria
Although the Kaggle competition link is not directly accessible here, commonly used evaluation metrics for regression problems include the Root Mean Squared Error (RMSE), Mean Absolute Error (MAE), and R-squared. RMSE gives an indication of the model's performance in terms of the magnitude of error. A lower RMSE indicates a better fit of the model to the data. For the sake of this exercise, let's assume RMSE is the chosen metric, as it penalizes large errors more than smaller ones, making it sensitive to outliers.

##### 3. Types of Variables/Features
The dataset comprises a mix of categorical, numerical, and boolean features. Here's a breakdown:

Categorical Features: body_type, city, engine_type, exterior_color, interior_color, listing_color, make_name, model_name, transmission, transmission_display, wheel_system.
Numerical Features: city_fuel_economy, daysonmarket, dealer_zip, engine_displacement, highway_fuel_economy, horsepower, latitude, longitude, mileage, savings_amount, seller_rating, year.
Boolean Features: franchise_dealer, has_accidents, is_new.
String Features with Quantitative Information: Many features like back_legroom, front_legroom, fueltankvolume, height, length, power, torque, wheelbase, width are provided as strings but contain numeric data (with units).
##### 4. Data Summary and Main Data Characteristics
The dataset contains information about various vehicles, including their technical specifications, physical attributes, condition, and listing details. The target variable is the price of the vehicle. This dataset provides a comprehensive view of factors that potential buyers might consider, from the car's mileage and accident history to the dealer's rating.

##### 5. Missing Values
Before discussing missing values, let's first identify which columns have them and how many they contain.

`(Task 1, Text Here - insert more cells as required)`

In [1]:
import pandas as pd

train_df = pd.read_csv('train.csv')

In [2]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   vin                   3500 non-null   object 
 1   back_legroom          3397 non-null   object 
 2   body_type             3494 non-null   object 
 3   city                  3500 non-null   object 
 4   city_fuel_economy     2912 non-null   float64
 5   daysonmarket          3500 non-null   int64  
 6   dealer_zip            3500 non-null   int64  
 7   engine_displacement   3375 non-null   float64
 8   engine_type           3450 non-null   object 
 9   exterior_color        3500 non-null   object 
 10  franchise_dealer      3500 non-null   bool   
 11  front_legroom         3397 non-null   object 
 12  fuel_tank_volume      3397 non-null   object 
 13  fuel_type             3463 non-null   object 
 14  height                3397 non-null   object 
 15  highway_fuel_economy 

In [3]:
train_df.isna().sum()

vin                       0
back_legroom            103
body_type                 6
city                      0
city_fuel_economy       588
daysonmarket              0
dealer_zip                0
engine_displacement     125
engine_type              50
exterior_color            0
franchise_dealer          0
front_legroom           103
fuel_tank_volume        103
fuel_type                37
height                  103
highway_fuel_economy    588
horsepower              125
interior_color            0
is_new                    0
latitude                  0
length                  103
listed_date               0
listing_color             0
longitude                 0
make_name                 0
maximum_seating         103
mileage                 203
model_name                0
power                   299
savings_amount            0
seller_rating             0
torque                  331
transmission             60
transmission_display     60
wheel_system            101
wheelbase           

In [4]:
# Check for missing values in the training data again
missing_values = train_df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

missing_values

city_fuel_economy       588
highway_fuel_economy    588
torque                  331
power                   299
mileage                 203
horsepower              125
engine_displacement     125
length                  103
wheelbase               103
maximum_seating         103
back_legroom            103
height                  103
fuel_tank_volume        103
front_legroom           103
width                   103
wheel_system            101
transmission             60
transmission_display     60
engine_type              50
fuel_type                37
body_type                 6
dtype: int64

Numeric Features: (e.g., `city_fuel_economy`, `highway_fuel_economy`, `torque`, `power`, `mileage`, `horsepower`, `engine_displacement`, `length`, `wheelbase`, etc.): For numeric features, we consider imputing missing values using statistical measures like the mean, median, or mode, depending on the distribution of the data. Imputing with the mean is a common approach and can be done for columns where the missing values are not too extensive. Categorical Features (e.g., `engine_type`, `fuel_type`, `body_type`, etc.): For categorical features, we impute missing values with the mode (most frequent category) or introduce a new category like "Other".

---

## Task 2: Data Cleaning, Missing Observations and Feature Engineering
- In this task you will follow a set of instructions/questions listed below.
- Make sure you explain each answer carefully both in Markdown text and on your video.

Total Marks: 12

**Task 2, Question 1**: Clean **all** numerical features so that they can be used in training algorithms. For instance, back_legroom feature is in object format containing both numerical values and text. Extract numerical values (equivalently eliminate the text) so that the numerical values can be used as a regular feature.  
(2 marks)

In [5]:
import pandas as pd
import numpy as np

In [6]:
def clean_features(data):
    # Features with units that need cleaning
    features_with_units = ['back_legroom', 'front_legroom', 'fuel_tank_volume', 'height', 'length', 'wheelbase', 'width', 'maximum_seating']

    # Extract numerical values from these features
    for feature in features_with_units:
        if data[feature].dtype == 'object':
            data[feature] = data[feature].str.extract('(\d+\.?\d*)').astype(float)
            
    # Display the cleaned features
    print(data[features_with_units].head())

In order to clean all the numerical features so that they can be used in training algorithms, we use the clean_feature function. The clean_feature function is designed to clean and preprocess a dataset, specifically focusing on certain features that have units. We first identifies a list of features that need cleaning. These features are 'back_legroom', 'front_legroom', 'fuel_tank_volume', 'height', 'length', 'wheelbase', 'width', and 'maximum_seating'. We check for if the data type of the feature is an object, if the feature is indeed an object, it applies a regular expression to extract the numerical part of the string. The regular expression (\d+.?\d*) is used to match any number that may or may not have a decimal point, then convert to the float data type.

**Task 2, Question 2** Create at least 5 new features from the existing numerical variables which contain multiple items of information, for example you could extract maximum torque and torque rpm from the torque variable.  
(2 marks)

In [7]:
def new_features(data):
    data['max_torque'] = pd.to_numeric(data['torque'].str.split().str[0])
    data['torque_rpm'] = data['torque'].str.split().str[3]
    data['torque_rpm'] = data['torque_rpm'].str.replace(',', '')
    data['torque_rpm'] = pd.to_numeric(data['torque_rpm'])
    
    data['power_value'] = pd.to_numeric(data['power'].str.split().str[0])
    data['power_rpm'] = data['power'].str.split().str[3]
    data['power_rpm'] = data['power_rpm'].str.replace(',', '')
    data['power_rpm'] = pd.to_numeric(data['power_rpm'])
    
    data['torque_to_power_ratio'] = data['max_torque'] / data['power_value']

Creates a new feature max_torque by splitting the torque feature on spaces and taking the first item (str[0]) (which is assumed to be the maximum torque value), and converting it to a numeric type. We create another new feature torque_rpm by splitting the torque feature on spaces and taking the fourth item (which is assumed to be the RPM at which maximum torque is produced). It removes any commas from this value and converts it to a numeric type, for example 12,000 in to 12000. Similarly,a power_value feature is created by splitting the power feature on spaces, taking the first item (which is assumed to be the power value), and converting it to a numeric type.

**Task 2, Question 3**: Impute missing values for all features in both the training and test datasets.   
(3 marks)

In [8]:
# Define a function to impute missing values
def impute_missing_values(df):
    for column in df.columns:
        # If column data type is object (categorical) then fill missing with mode
        if df[column].dtype == 'object':
            df[column].fillna(df[column].mode()[0], inplace=True)
        # If column data type is numerical or boolean then fill missing with median
        else:
            df[column].fillna(df[column].median(), inplace=True)
    return df

Creates a new feature max_torque by splitting the torque feature on spaces and taking the first item (str[0]) (which is assumed to be the maximum torque value), and converting it to a numeric type. We create another new feature torque_rpm by splitting the torque feature on spaces and taking the fourth item (which is assumed to be the RPM at which maximum torque is produced). It removes any commas from this value and converts it to a numeric type, for example 12,000 in to 12000. Similarly,a power_value feature is created by splitting the power feature on spaces, taking the first item (which is assumed to be the power value), and converting it to a numeric type.

**Task 2, Question 4**: Encode all categorical variables appropriately as discussed in class. 

- Where multiple values are given for an observation encode the observation as 'other'. 
- Where a categorical feature contains more than 5 unique values, map the features into 5 most frequent values + 'other' and then encode appropriately. For instance, map colours into 5 basic colours + 'other': [red, yellow, green, blue, purple, other] and then encode.  
(2 marks)

In [9]:
# Define a function to encode categorical variables
def encode_categorical_columns(df):
    for column in df.select_dtypes(include=['object', 'bool']).columns:
        # Set observations with multiple values to 'other'
        df[column] = df[column].apply(lambda x: 'other' if '/' in str(x) else x)
        
        if df[column].dtype == bool:
            df[column] = df[column].astype(int)
            
        # If there are more than 5 unique values, keep the top 5 and set the rest to 'other'
        if df[column].nunique() > 5 and column != 'vin':
            top_5_values = df[column].value_counts().head(5).index
            #print(df[column].value_counts().head(5))
            #print(top_5_values)
            df[column] = df[column].apply(lambda x: x if x in top_5_values else 'other')
    
    # One-hot encode the categorical columns
    dummies = pd.get_dummies(df.drop('vin', axis = 1), drop_first=True)

    # Add the excluded column back to the dummies DataFrame
    dummies['vin'] = df['vin']

    return dummies

A copy of the original DataFrame `car` is created and stored in `car_graph` for reference. Several columns, including 'power', 'torque', 'vin', 'latitude', 'longitude', and 'listed_date', are dropped from the `car` DataFrame. These columns are likely removed because they may not be relevant to the analysis or modeling task. The 'dealer_zip' column is converted to a string data type, possibly to treat it as a categorical variable during encoding. A function called `encode_categorical_columns` is defined to encode categorical variables. The function performs the following operations for each categorical column:
   - Replaces values containing '/' with 'other'.
   - Converts boolean values to integers (1 for True, 0 for False).
   - Keeps the top 5 most frequent values in the column and replaces the rest with 'other'.
   - Performs one-hot encoding, dropping the first category to avoid multicollinearity. 
   
The `encode_categorical_columns` function is applied to the `car` DataFrame, and the result is stored in the `car_encoded` DataFrame. This DataFrame contains one-hot encoded features for the categorical variables. The code displays the first few rows of the `car_encoded` DataFrame to provide a preview of the transformed data.

**Task 2, Question 5**: Perform any other actions you think need to be done on the data before constructing predictive models, and clearly explain what you have done.   
(1 marks)

##### 1. Feature Scaling:
Machine learning algorithms perform better when numerical input variables are scaled to a standard range. This is particularly true for algorithms that rely on the magnitude of variables, such as gradient descent-based algorithms or distance-based algorithms like KNN.

StandardScaler is a common method to scale features, which will transform the data such that its distribution has a mean value of 0 and a standard deviation of 1.

##### 2. Handling Outliers:
Outliers can adversely affect the performance of some models. We can use methods like the IQR (Inter-Quartile Range) to identify and remove outliers from the dataset.

##### 3. Feature Engineering:
Given the nature of the dataset, there might be some potential interactions between features that could be useful. For instance, creating a feature that combines city and highway fuel economy might be indicative.

##### 4. Reducing Dimensionality:
After one-hot encoding, the number of features in the dataset may increase significantly. Some of these features might be highly correlated, and using them all might lead to overfitting. Methods like PCA (Principal Component Analysis) can be employed to reduce the dimensionality.

In [10]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
merged_df = pd.merge(train_df.drop(columns=['price']), test_df, how='outer')

clean_features(merged_df)
new_features(merged_df)
impute_missing_values(merged_df)

merged_ori = merged_df.copy()
merged_df = merged_df.drop(['power', 'torque', 'listed_date'], axis = 1)
merged_df['dealer_zip'] = merged_df['dealer_zip'].apply(str)
merged_encoded = encode_categorical_columns(merged_df)

   back_legroom  front_legroom  fuel_tank_volume  height  length  wheelbase  \
0          33.5           41.3              14.8    60.2   174.2      106.3   
1          36.8           42.8              15.5    65.6   179.2      105.9   
2          36.8           42.8              15.5    65.6   179.2      105.9   
3          30.3           42.6              13.2    55.0   175.5      104.3   
4          38.6           43.2              16.2    64.1   180.6      106.7   

   width  maximum_seating  
0   82.0              5.0  
1   84.1              5.0  
2   84.1              5.0  
3   68.9              5.0  
4   83.0              5.0  


In [11]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

columns_to_scale = merged_encoded.select_dtypes(include=[np.number]).columns.tolist()

# Fit the scaler to the selected columns and transform the data

merged_encoded_sc = merged_encoded.copy()
merged_encoded_sc[columns_to_scale] = sc.fit_transform(merged_encoded_sc[columns_to_scale])

In [12]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler 
scaler_encoded = MinMaxScaler()

# Apply MinMax scaling directly to the identified numerical columns in the train and test datasets
for col in columns_to_scale:
    # Fit the scaler to the training data column and transform
    merged_encoded[col] = scaler_encoded.fit_transform(merged_encoded[[col]])

In [13]:
from sklearn.preprocessing import PolynomialFeatures

quadratic = PolynomialFeatures(degree=2)
quadratic_features = quadratic.fit_transform(merged_encoded[['horsepower']])

# create a new column 'horsepower_squared' with the quadratic features
merged_encoded['horsepower_squared'] = quadratic_features[:, 1]

merged_encoded

,back_legroom,city_fuel_economy,daysonmarket,engine_displacement,franchise_dealer,front_legroom,fuel_tank_volume,height,highway_fuel_economy,horsepower,...,transmission_display_8-Speed Automatic,transmission_display_Automatic,transmission_display_Continuously Variable Transmission,transmission_display_other,wheel_system_4X2,wheel_system_AWD,wheel_system_FWD,wheel_system_RWD,vin,horsepower_squared
0,0.705263,0.094017,0.015974,0.166667,0.0,0.176282,0.148718,0.231392,0.137615,0.180807,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,SJKCH5CRXHA032566,0.180807
1,0.774737,0.076923,0.051118,0.216667,1.0,0.224359,0.166667,0.318770,0.110092,0.273992,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5LMCJ3D96HUL54638,0.273992
2,0.774737,0.076923,0.011182,0.216667,1.0,0.224359,0.166667,0.318770,0.110092,0.273992,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5LMCJ2D95HUL35217,0.273992
3,0.637895,0.128205,0.010383,0.133333,0.0,0.217949,0.107692,0.147249,0.211009,0.086231,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2HGFG1B86AH500600,0.086231
4,0.812632,0.094017,0.011182,0.166667,1.0,0.237179,0.184615,0.294498,0.146789,0.239221,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5LMCJ1D95LUL25032,0.239221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.709474,0.076923,0.015974,0.333333,0.0,0.192308,0.182051,0.132686,0.128440,0.308762,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,WBAKF9C53CE672450,0.308762
4996,0.751579,0.136752,0.190096,0.066667,1.0,0.160256,0.128205,0.325243,0.165138,0.083449,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,KL7CJLSB1LB063169,0.083449
4997,0.755789,0.076923,0.006390,0.416667,0.0,0.205128,0.243590,0.168285,0.137615,0.264256,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,JTHBK1EG7C2488661,0.264256
4998,0.732632,0.145299,0.022364,0.250000,1.0,0.330128,0.176923,0.177994,0.220183,0.157163,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5NPEF4JA3LH058405,0.157163


We apply feature scaling to the numerical features in the training data (`train_df_encoded`) using the `fit_transform` method. `train_df_encoded` is assumed to be a DataFrame containing both numerical and categorical columns. `common_numerical_cols` is expected to be a list of column names that correspond to the numerical features. This means that the numerical columns in `train_df_encoded` will be standardized. 

   The `fit_transform` method does two things:
   - It computes the mean and standard deviation of each numerical column in the `common_numerical_cols` from the training data.
   - It then standardizes the data by subtracting the mean and dividing by the standard deviation.

Then apply the same scaler to transform the numerical features in the test data (`test_df_encoded`) using the `transform` method. This ensures that the scaling applied to the test data is consistent with the scaling used on the training data. This step is crucial to avoid data leakage and ensure that the model can be tested on unseen data.

After running this code, numerical features in both the training and test datasets will be standardized, with a mean of 0 and a standard deviation of 1. This normalization can help improve the performance and stability of various machine learning algorithms when applied to the data.


**Task 2, Question 6**: Perform exploratory data analysis to measure the relationship between the features and the target and carefully write up your findings. 
(2 marks)

In [14]:
! pip install folium

import folium

# Create a map centered on a specific latitude and longitude
m = folium.Map(location=[train_df_graph['latitude'].iloc[0], train_df_graph['longitude'].iloc[0]], zoom_start=10)

# Add markers for each point
for index, row in train_df_graph.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['dealer_zip']).add_to(m)

# Save the map to an HTML file or display it
m.save('map.html')

ModuleNotFoundError: No module named 'folium'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

features = ['back_legroom', 'city_fuel_economy', 'daysonmarket', 'engine_displacement', 'front_legroom',
                            'fuel_tank_volume', 'height', 'highway_fuel_economy', 'horsepower',
                            'length', 'maximum_seating', 'mileage',
                            'savings_amount', 'seller_rating', 'wheelbase', 'width',
                            'max_torque', 'torque_rpm', 'power_value', 'power_rpm']

# Extracting the correlation values of the corrected features with 'price'
correlation_matrix = train_df_encoded[['price'] + features].corr()

# Plotting the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, fmt=".2f")
plt.title('Correlation Heatmap of Corrected Features with Price')
plt.show()


In [ ]:
import seaborn as sns

# Create a pairplot
sns.pairplot(train_df_encoded[['back_legroom', 'city_fuel_economy', 'daysonmarket', 'engine_displacement', 'front_legroom',
                            'fuel_tank_volume', 'height', 'highway_fuel_economy', 'horsepower']])


# Show the plot
plt.show()

In [ ]:
'length', 'maximum_seating', 'mileage',
                            'savings_amount', 'seller_rating', 'wheelbase', 'width',
                            'max_torque', 'torque_rpm', 'power_value', 'power_rpm'

Visualize the Distribution of the Target Variable (price): A histogram or kernel density plot can help us understand the distribution of the target variable.

Correlation Analysis: We'll compute the correlation of each feature with the target variable to identify which features are most strongly associated with the price.

Visualize Relationships for Key Features: For the top features (based on correlation), we'll visualize their relationship with the target variable using scatter plots or boxplots.

Visualize Categorical Features: For key categorical features, we can use boxplots to understand the distribution of the target variable across different categories.

--- 
## Task 3: Fit and tune a forecasting model/Submit predictions/Report score and ranking

Make sure you **clearly explain each step** you do both in text and on the recoded video.   
This task must not create any additional features and has to use on the dataset constructed in Task 2.

1. Build at least 3 machine learning (ML) regression models taking into account the outcomes of Tasks 1 & 2 (Explain Carefully)
2. Fit the models and tune hyperparameters via cross-validation: make sure you comment and explain each step clearly
3. Select your best algorithm, create predictions using the test dataset, and submit your predictions on Kaggle's competition page
4. Provide Kaggle ranking and **score** (screenshot your best submission) and comment
5. Make sure your Python code works, so that a marker that can replicate your all Kaggle Score   

- Hint: to perform well you will need to iterate Tasks 2 and Task 3.

Total Marks: 12

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

In [ ]:
X = merged_encoded[merged_encoded['vin'].isin(train_df['vin'])].drop('vin', axis=1).values

y = train_df['price'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
models = {
    #'Linear Regression': LinearRegression(),
    #'Random Forest': RandomForestRegressor(random_state=42),
    #'XGBoost': xgb.XGBRegressor(random_state=42),
    #'Ridge Regression': Ridge(random_state=42),
    #'Gradient Boosted Regression Trees': GradientBoostingRegressor(random_state=42),
    #'Kernel Ridge Regressor' : KernelRidge(random_state=42),
    #'Support Vector Regressor' : SVR(random_state=42),
    #'Bagging Regressor' : BaggingRegressor(random_state=42),
    #'Adaptive Boost Regressor' : AdaBoostRegressor(random_state=42),
    #'Multi-layer Perceptron Regressor' : MLPRegressor(random_state=42)
}

# Define the parameters for grid search
params = {
    #'Linear Regression': { },
    #'Random Forest': { 'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [2, 5, 10] },
    #'XGBoost': { 'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7, 9], 'alpha': [0.1, 0.5, 1.0, 5.0, 10.0] },
    #'Ridge Regression': { 'alpha': [0.1, 0.5, 1.0, 5.0, 10.0] },
    #'Gradient Boosted Regression Trees': {
    #'n_estimators': [50, 100, 250],
    #'learning_rate': [0.01, 0.05, 0.15],
    #'max_depth': [3, 4, 7, 9],
    #'min_samples_split': [2, 5, 10],
    #'min_samples_leaf': [2, 5, 10],
    #'subsample': [0.8, 0.9, 1.0],
    #'max_features': ['sqrt', 0.8],
    #'Kernel Ridge Regressor': {'alpha': [0.1, 1.0, 10.0],'kernel': ['linear', 'polynomial', 'rbf'], 'gamma': [0.1, 1.0, 10.0]},
    #'Support Vector Regressor': {'kernel': ['linear', 'rbf', 'poly'], 'C': [0.1, 1.0, 10.0], 'gamma': [0.1, 1.0, 10.0]},
    #'Bagging Regressor': {'n_estimators': [10, 50, 100], 'max_samples': [0.5, 1.0]},
    #'Adaptive Boost Regressor': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1.0]},
    #'Multi-layer Perceptron Regressor': {'hidden_layer_sizes': [(50,), (100, 50), (100, 100, 50)], 'activation': ['relu', 'tanh']},
}

# Initialize an empty dictionary to store the best models
best_models = {}

# Perform grid search
for model_name in models:
    model = models[model_name]
    param_grid = params[model_name]
    random_search = RandomizedSearchCV(model, param_grid, n_iter=10, scoring='neg_mean_squared_error', cv=KFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1)
    random_search.fit(X_train, y_train)
    
    best_models[model_name] = random_search.best_estimator_
    
    print(f"Best parameters for {model_name}: {random_search.best_params_}")
    print(f'Best model for {model_name}: {random_search.best_estimator_}')
    print(f"Best {model_name} cross-validation score (RMSE): {np.sqrt(np.abs(random_search.best_score_))}")
    print("\n")

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Evaluate the best models on the test set
for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    rmse = r2_score(y_test, y_pred)
    print(f"{model_name} Test RMSE: {rmse}")

## Searching for the best hyperparatermers for each model

### Linear regression

- Best parameters for Linear Regression: {}
- Best Linear Regression cross-validation score (MSE): 85402556.17530693
- Linear Regression Test MSE: 125657738.31148525

### Random Forest

Two hyperparameter sets were identified from two gridsearches, one with cv = 5, the other with cv=KFold(n_splits=5, shuffle=True, random_state=42)

- Best parameters for Random Forest: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
- Best Random Forest cross-validation score (MSE): 36089024.152263604
- Random Forest Test MSE: 67024115.424000144

- Best parameters for Random Forest (KFold): {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}


### XG Boost

Two hyperparameter sets were identified from two gridsearches, one with cv = 5, the other with cv=KFold(n_splits=5, shuffle=True, random_state=42)

- Best parameters for XGBoost: {'alpha': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
- Best XGBoost cross-validation score (MSE): 33958151.307104185
- XGBoost Test MSE: 71178404.95253536

- Best parameters for XGBoost (KFold): {'alpha': 1, 'learning_rate': 0.05, 'max_depth': 9, 'n_estimators': 200}


### Gradient boosted regression trees
- Best parameters for Gradient Boosted Regression Trees: {'learning_rate': 0.1, 'max_depth': 7, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
- Best Gradient Boosted Regression Trees cross-validation score (MSE): 34216888.000975125

- Best parameters for Gradient Boosted Regression Trees (KFold): {'learning_rate': 0.1, 'max_depth': 7, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
- Best Gradient Boosted Regression Trees cross-validation score (MSE): 28012239.107196353
- Gradient Boosted Regression Trees Test MSE: 76421709.90472016


### Bagging regressor

- Best parameters for Bagging Regressor: {'max_samples': 1.0, 'n_estimators': 100}
- Best Bagging Regressor cross-validation score (MSE): 42958938.22874457


- Best parameters for Bagging Regressor: {'max_samples': 1.0, 'n_estimators': 50}
- Best Bagging Regressor cross-validation score (MSE): 27744856.929287888
- Bagging Regressor Test MSE: 77610694.74220997

### Other models

Because GridSearchCV takes a really long time to run, I wasn't able to run the whole cell in one go, instead I have to pick some models for each run.

I also have to do GridSearchCV on a simpler training dataset (with only the top 20 variables included in training)

From a first pass, I elimited the following models because they clearly performed worse than the other models:
- 'Ridge Regression': Ridge(),
- 'Kernel Ridge Regressor' : KernelRidge(),
- 'Support Vector Regressor' : SVR(),
- 'Adaptive Boost Regressor' : AdaBoostRegressor(),
- 'Multi-layer Perceptron Regressor' : MLPRegressor()

Best parameters for Ridge Regression: {'alpha': 5.0}
Best model for Ridge Regression: Ridge(alpha=5.0)
Best Ridge Regression cross-validation score (RMSE): 8712.027317339205


Best parameters for Kernel Ridge Regressor: {'kernel': 'linear', 'gamma': 10.0, 'alpha': 0.1}
Best model for Kernel Ridge Regressor: KernelRidge(alpha=0.1, gamma=10.0)
Best Kernel Ridge Regressor cross-validation score (RMSE): 8874.359661958888


## Fitting the best models parameters found from Gridsearch and Randomsearch

This part I will fit the chosen models from GridSearchCV and RandomSearch to the whole training set, and calculate the RMSE of each model to see which one does better.

In [ ]:
models = {
    'Random Forest 1': RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=5, min_samples_leaf = 2, random_state=42),
    'Random Forest 2': RandomForestRegressor(n_estimators=50, max_depth=None, min_samples_split=5, min_samples_leaf = 2, random_state=42),
    'XG Boost 1': xgb.XGBRegressor(alpha=1, learning_rate=0.05, max_depth=9, n_estimators=200, random_state=42),
    'XG Boost 2': xgb.XGBRegressor(alpha=0.1, learning_rate=0.1, max_depth=5, n_estimators=200, random_state=42),
    'Gradient Boosted 1' : GradientBoostingRegressor(learning_rate=0.1, max_depth=7, min_samples_leaf=2, min_samples_split=10,
                              n_estimators=200, random_state=42),
    'Gradient Boosted 2' : GradientBoostingRegressor(learning_rate=0.15, max_depth=4, max_features=0.8,
                          min_samples_leaf=2, min_samples_split=6,
                          n_estimators=250, random_state=42, subsample=0.8),
    'Bagging Regressor' : BaggingRegressor(max_samples=1, n_estimators=100)
}

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

def fit_model(model, X, X_test, y, y_test):
    model.fit(X, y)
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)

    print(f'RMSE train: {r2_score(y_train, pred_train):.3f}, test: {r2_score(y_test, pred_test):.4f}')

In [ ]:
for model_name in models:
    model = models[model_name]
    print(model_name)
    fit_model(model, X_train, X_test, y_train, y_test)

## A quick look

First having a quick look at all the models:

RMSE for each model

Random Forest 1
RMSE train: 0.978, test: 0.8030

Random Forest 2
RMSE train: 0.976, test: 0.8042

XG Boost 1
RMSE train: 0.996, test: 0.8012

XG Boost 2
RMSE train: 0.986, test: 0.8235

Gradient Boosted 1
RMSE train: 0.996, test: 0.8181

Gradient Boosted 2
RMSE train: 0.987, test: 0.8347

Bagging Regressor
RMSE train: -0.003, test: -0.0240

We can see that basically the models perform with the full set of features, and there is a superior Random forest and XGBoost and Gradient Boosted hyperparameter set.

Out of all models, Gradient boosted 2 is performing the best.

In [ ]:
gb3 = GradientBoostingRegressor(learning_rate=0.15, max_depth=4, max_features=0.8,
                          min_samples_leaf=2, min_samples_split=6,
                          n_estimators=250, random_state=42, subsample=0.8)

fit_model(gb3, X, X_test, y, y_test)

In [ ]:
X_test_merged = merged_encoded[merged_encoded['vin'].isin(test_df['vin'])].drop('vin', axis=1)

y_pred_merged_gb3 = gb3.predict(X_test_merged)

In [ ]:
test_df['price'] = y_pred_merged_gb3

test_df[['vin', 'price']].to_csv('submission_gb3_merged_minmax_quad.csv', index=False)

In [ ]:
import pickle
pickle.dump(gb3, open("gb3_merged.pkl", "wb"))

## Stacking Regressor

This part I try to use stacking regressor with the two best performing models. It looks like the model does well on the training set, but not too well on test set. It could be overfitting to the training set.

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

gb3 = GradientBoostingRegressor(learning_rate=0.15, max_depth=4, max_features=0.8,
                          min_samples_leaf=2, min_samples_split=6,
                          n_estimators=250, random_state=42, subsample=0.8)

xg2 = xgb.XGBRegressor(alpha=0.1, learning_rate=0.1, max_depth=5, n_estimators=200, random_state=42)

estimators = [('Gradient Boosted',  gb3), ('XGBoost', xg2)]
            
stacking_reg = StackingRegressor(estimators=estimators, final_estimator = DecisionTreeRegressor(random_state =42))

stacking_reg.fit(X_train, y_train)

y_train_pred_stacking = stacking_reg.predict(X_train)
y_test_pred_stacking = stacking_reg.predict(X_test)


print(f'RMSE train: {r2_score(y_train, y_train_pred_stacking):.3f}, test: {r2_score(y_test, y_test_pred_stacking):.3f}')

## Feature importance

This part helps to illustrate the most influential features in the model using Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

rf = RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=5, min_samples_leaf = 2, random_state=42)

# Train a Random Forest Regressor
rf.fit(X_train, y_train)

# Get feature importances from the trained model
feature_importances = rf.feature_importances_

# Create a DataFrame to display feature names and their importance scores
train = train_df_encoded.copy().drop(['price'], axis = 1)
feature_names = train.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# Sort the DataFrame by importance scores in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Create a bar plot to visualize feature importances
plt.figure(figsize=(10, 20))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importance in Random Forest Regressor')
plt.gca().invert_yaxis()  # Invert the y-axis to show the most important features at the top
plt.show()

# Display the sorted feature importances
print(importance_df)

## PCA, KPCA and LDA

This part uses feature extraction to see if MSE is improved by reducing the dimensionality of the dataset.

It looks like none of the methods helped to improve MSE.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 13)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

gb3.fit(X_train_pca, y_train)

gb3_pca_pred = gb3.predict(X_test_pca)

gb3_pca_rmse = r2_score(y_test, gb3_pca_pred)

print(gb3_pca_rmse)

In [ ]:
from sklearn.decomposition import KernelPCA

kpca = KernelPCA(n_components=13, kernel='rbf', gamma=15)
X_train_kpca = kpca.fit_transform(X_train)
X_test_kpca = kpca.transform(X_test)

gb3.fit(X_train_kpca, y_train)

gb3_kpca_pred = gb3.predict(X_test_kpca)

gb3_kpca_rmse = r2_score(y_test, gb3_kpca_pred)

print(gb3_kpca_rmse)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components = 13)

X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

gb3.fit(X_train_lda, y_train)

gb3_lda_pred = gb3.predict(X_test_lda)

gb3_lda_rmse = r2_score(y_test, gb3_lda_pred)

print(gb3_lda_rmse)

## Final conclusions

We can see that XGBoost on merged dataset did the best on the test set.

Eventhough Gradient Boosted performs slightly better on training set, it is not as good as XGBoost on the test set.